# Pulse Length for H2 is 4.75 ns with Flexible Partial Compilation

(should take max-per-slice over more runs though)

In [2]:
import sys
sys.path.append('../..')
import config
from fqc import uccsd, util

import numpy as np
from datetime import datetime

data_path = config.DATA_PATH
file_name = datetime.today().strftime('%h%d')

In [3]:
from quantum_optimal_control.helper_functions.grape_functions import transmon_gate
from quantum_optimal_control.main_grape.grape import Grape
from quantum_optimal_control.core import hamiltonian

In [4]:
d = 2  # this is the number of energy levels to consider (i.e. d-level qudits)
max_iterations = 6000
decay =  max_iterations / 2
convergence = {'rate':0.01, 'max_iterations': max_iterations,
               'conv_target':1e-3, 'learning_rate_decay':decay, 'min_grad': 1e-12, 'update_step': 20}
reg_coeffs = {}

In [5]:
N = 2
connected_qubit_pairs = util.get_nearest_neighbor_coupling_list(1, 2, directed=False)
H0 = hamiltonian.get_H0(N, d)
Hops, Hnames = hamiltonian.get_Hops_and_Hnames(N, d, connected_qubit_pairs)
states_concerned_list = hamiltonian.get_full_states_concerned_list(N, d)
maxA = hamiltonian.get_maxA(N, d, connected_qubit_pairs)

In [6]:
circuit = uccsd.get_uccsd_circuit('H2')

In [7]:
    slices = uccsd.get_uccsd_slices(circuit, granularity=2, dependence_grouping=True)
for slice in slices:
    print(slice.circuit.draw(output='text', line_length=10000))
    print('\n---------------------\n')

        ┌───┐┌────────────┐┌───┐┌───────────┐
q_0: |0>┤ X ├┤ Rx(1.5708) ├┤ X ├┤ Rz(7.387) ├
        └───┘└────────────┘└───┘└───────────┘
q_1: |0>─────────────────────────────────────
                                             

---------------------

                                    ┌───┐┌─────────────┐
q_0: |0>────────────────────────────┤ X ├┤ Rx(-1.5708) ├
        ┌────────────┐┌────────────┐└───┘└─────────────┘
q_1: |0>┤ Rx(1.5708) ├┤ Rz(10.562) ├────────────────────
        └────────────┘└────────────┘                    

---------------------

                            ┌────────────┐                                                     ┌─────────────┐┌───┐                                       ┌───┐
q_0: |0>────────────────────┤ Rx(1.5708) ├──■────────────────────────────■─────────────────────┤ Rx(-1.5708) ├┤ H ├──■──────────────────■─────────────────┤ H ├
        ┌─────────────┐┌───┐└────────────┘┌─┴─┐┌───┐┌────────────┐┌───┐┌─┴─┐┌───┐┌────────────┐└─────────────┘└───┘┌─

In [8]:
def binary_search_for_shortest_pulse_time(min_time, max_time, tolerance=1):
    """Search between [min_time, max_time] up to 1ns tolerance. Assumes 20 steps per ns."""
    min_steps, max_steps = min_time * 20, max_time * 20
    while min_steps + 20 * tolerance < max_steps:  # just estimate to +- 1ns
        mid_steps = int((min_steps + max_steps) / 2)
        total_time = mid_steps / 20.0
        print('\n\ntrying total_time: %s for unitary of size %s' % (str(total_time), str(U.shape)))
        SS = Grape(H0, Hops, Hnames, U, total_time, mid_steps, states_concerned_list, convergence,
                         reg_coeffs=reg_coeffs,
                         use_gpu=False, sparse_H=False, method='Adam', maxA=maxA,
                         show_plots=False, file_name=file_name, data_path=data_path)
        if SS.l < SS.conv.conv_target:  # if converged, search lower half
            max_steps = mid_steps
        else:
            min_steps = mid_steps

    return mid_steps / 20

In [15]:
times = []
for slice in slices:
    U = slice.unitary()
    times.append(binary_search_for_shortest_pulse_time(0.0, 10.0, tolerance=0.3))



trying total_time: 5.0 for unitary of size (4, 4)
data saved at: /project/ftchong/qoc/pranav/00029_Mar28.h5
Using 6 Taylor terms and 1 Scaling & Squaring terms
Building graph:
Propagators initialized.
Operators weight initialized.
Intermediate propagation variables initialized.
Intermediate propagators initialized.
Vectors initialized.
Training loss initialized.
Optimizer initialized.
Utilities initialized.
Graph built!
Initialized
Error = :9.06e-01; Runtime: 0.1s; Iterations = 0, grads =   4.387e-01, unitary_metric = 1.00000
Error = :1.51e-01; Runtime: 1.1s; Iterations = 20, grads =   2.537e-01, unitary_metric = 1.00000
Error = :8.90e-03; Runtime: 1.8s; Iterations = 40, grads =   1.218e-02, unitary_metric = 1.00000
Error = :4.43e-04; Runtime: 2.3s; Iterations = 56, grads =   1.110e-03, unitary_metric = 1.00000
data saved at: /project/ftchong/qoc/pranav/00029_Mar28.h5


trying total_time: 2.5 for unitary of size (4, 4)
data saved at: /project/ftchong/qoc/pranav/00030_Mar28.h5
Using 6

Error = :3.17e-03; Runtime: 12.1s; Iterations = 1200, grads =   1.178e-07, unitary_metric = 1.00000
Error = :3.16e-03; Runtime: 12.4s; Iterations = 1220, grads =   1.102e-07, unitary_metric = 1.00000
Error = :3.14e-03; Runtime: 12.6s; Iterations = 1240, grads =   1.032e-07, unitary_metric = 1.00000
Error = :3.13e-03; Runtime: 12.8s; Iterations = 1260, grads =   9.685e-08, unitary_metric = 1.00000
Error = :3.12e-03; Runtime: 13.0s; Iterations = 1280, grads =   9.096e-08, unitary_metric = 1.00000
Error = :3.10e-03; Runtime: 13.1s; Iterations = 1300, grads =   8.553e-08, unitary_metric = 1.00000
Error = :3.09e-03; Runtime: 13.4s; Iterations = 1320, grads =   8.052e-08, unitary_metric = 1.00000
Error = :3.08e-03; Runtime: 13.6s; Iterations = 1340, grads =   7.587e-08, unitary_metric = 1.00000
Error = :3.07e-03; Runtime: 13.8s; Iterations = 1360, grads =   7.156e-08, unitary_metric = 1.00000
Error = :3.07e-03; Runtime: 14.0s; Iterations = 1380, grads =   6.755e-08, unitary_metric = 1.00000


Error = :2.90e-03; Runtime: 28.6s; Iterations = 2840, grads =   4.519e-09, unitary_metric = 1.00000
Error = :2.90e-03; Runtime: 28.7s; Iterations = 2860, grads =   4.440e-09, unitary_metric = 1.00000
Error = :2.90e-03; Runtime: 28.9s; Iterations = 2880, grads =   4.363e-09, unitary_metric = 1.00000
Error = :2.90e-03; Runtime: 29.1s; Iterations = 2900, grads =   4.289e-09, unitary_metric = 1.00000
Error = :2.90e-03; Runtime: 29.4s; Iterations = 2920, grads =   4.218e-09, unitary_metric = 1.00000
Error = :2.90e-03; Runtime: 29.6s; Iterations = 2940, grads =   4.149e-09, unitary_metric = 1.00000
Error = :2.89e-03; Runtime: 29.7s; Iterations = 2960, grads =   4.084e-09, unitary_metric = 1.00000
Error = :2.89e-03; Runtime: 29.9s; Iterations = 2980, grads =   4.020e-09, unitary_metric = 1.00000
Error = :2.89e-03; Runtime: 30.1s; Iterations = 3000, grads =   3.958e-09, unitary_metric = 1.00000
Error = :2.89e-03; Runtime: 30.4s; Iterations = 3020, grads =   3.899e-09, unitary_metric = 1.00000


Error = :2.90e-03; Runtime: 44.9s; Iterations = 4480, grads =   1.929e-09, unitary_metric = 1.00000
Error = :2.90e-03; Runtime: 45.1s; Iterations = 4500, grads =   1.879e-09, unitary_metric = 1.00000
Error = :2.90e-03; Runtime: 45.4s; Iterations = 4520, grads =   1.859e-09, unitary_metric = 1.00000
Error = :2.90e-03; Runtime: 45.5s; Iterations = 4540, grads =   1.843e-09, unitary_metric = 1.00000
Error = :2.90e-03; Runtime: 45.7s; Iterations = 4560, grads =   1.828e-09, unitary_metric = 1.00000
Error = :2.90e-03; Runtime: 45.9s; Iterations = 4580, grads =   1.815e-09, unitary_metric = 1.00000
Error = :2.90e-03; Runtime: 46.1s; Iterations = 4600, grads =   1.802e-09, unitary_metric = 1.00000
Error = :2.90e-03; Runtime: 46.4s; Iterations = 4620, grads =   1.789e-09, unitary_metric = 1.00000
Error = :2.90e-03; Runtime: 46.5s; Iterations = 4640, grads =   1.776e-09, unitary_metric = 1.00000
Error = :2.90e-03; Runtime: 46.7s; Iterations = 4660, grads =   1.763e-09, unitary_metric = 1.00000


Error = :8.19e-01; Runtime: 0.1s; Iterations = 0, grads =   2.220e-01, unitary_metric = 1.00000
Error = :4.00e-01; Runtime: 0.5s; Iterations = 20, grads =   7.398e-02, unitary_metric = 1.00000
Error = :2.05e-01; Runtime: 0.8s; Iterations = 40, grads =   1.892e-02, unitary_metric = 1.00000
Error = :6.97e-02; Runtime: 1.0s; Iterations = 60, grads =   3.668e-03, unitary_metric = 1.00000
Error = :2.02e-02; Runtime: 1.3s; Iterations = 80, grads =   8.137e-04, unitary_metric = 1.00000
Error = :5.33e-03; Runtime: 1.6s; Iterations = 100, grads =   2.366e-04, unitary_metric = 1.00000
Error = :1.38e-03; Runtime: 1.9s; Iterations = 120, grads =   4.962e-05, unitary_metric = 1.00000
Error = :9.77e-04; Runtime: 2.0s; Iterations = 125, grads =   3.033e-05, unitary_metric = 1.00000
data saved at: /project/ftchong/qoc/pranav/00032_Mar28.h5


trying total_time: 1.55 for unitary of size (4, 4)
data saved at: /project/ftchong/qoc/pranav/00033_Mar28.h5
Using 6 Taylor terms and 1 Scaling & Squaring terms
B

...

Error = :3.94e-03; Runtime: 59.0s; Iterations = 4160, grads =   3.449e-08, unitary_metric = 1.00000
Error = :3.94e-03; Runtime: 59.3s; Iterations = 4180, grads =   3.395e-08, unitary_metric = 1.00000
Error = :3.93e-03; Runtime: 59.6s; Iterations = 4200, grads =   3.343e-08, unitary_metric = 1.00000
Error = :3.93e-03; Runtime: 59.9s; Iterations = 4220, grads =   3.291e-08, unitary_metric = 1.00000
Error = :3.93e-03; Runtime: 60.2s; Iterations = 4240, grads =   3.242e-08, unitary_metric = 1.00000
Error = :3.93e-03; Runtime: 60.4s; Iterations = 4260, grads =   3.194e-08, unitary_metric = 1.00000
Error = :3.92e-03; Runtime: 60.7s; Iterations = 4280, grads =   3.148e-08, unitary_metric = 1.00000
Error = :3.92e-03; Runtime: 61.0s; Iterations = 4300, grads =   3.102e-08, unitary_metric = 1.00000
Error = :3.92e-03; Runtime: 61.3s; Iterations = 4320, grads =   3.059e-08, unitary_metric = 1.00000
Error = :3.92e-03; Runtime: 61.6s; Iterations = 4340, grads =   3.016e-08, unitary_metric = 1.00000


Error = :3.77e-03; Runtime: 82.2s; Iterations = 5800, grads =   1.481e-08, unitary_metric = 1.00000
Error = :3.77e-03; Runtime: 82.5s; Iterations = 5820, grads =   1.470e-08, unitary_metric = 1.00000
Error = :3.77e-03; Runtime: 82.8s; Iterations = 5840, grads =   1.458e-08, unitary_metric = 1.00000
Error = :3.77e-03; Runtime: 83.0s; Iterations = 5860, grads =   1.447e-08, unitary_metric = 1.00000
Error = :3.77e-03; Runtime: 83.3s; Iterations = 5880, grads =   1.436e-08, unitary_metric = 1.00000
Error = :3.77e-03; Runtime: 83.6s; Iterations = 5900, grads =   1.426e-08, unitary_metric = 1.00000
Error = :3.76e-03; Runtime: 83.9s; Iterations = 5920, grads =   1.415e-08, unitary_metric = 1.00000
Error = :3.76e-03; Runtime: 84.2s; Iterations = 5940, grads =   1.404e-08, unitary_metric = 1.00000
Error = :3.76e-03; Runtime: 84.5s; Iterations = 5960, grads =   1.394e-08, unitary_metric = 1.00000
Error = :3.76e-03; Runtime: 84.7s; Iterations = 5980, grads =   1.383e-08, unitary_metric = 1.00000


Process Process-458:
Process Process-464:
Process Process-474:
Process Process-465:
Process Process-472:
Process Process-459:
Process Process-483:
Process Process-477:
Process Process-471:
Process Process-470:
Process Process-480:
Process Process-481:
Process Process-479:
Process Process-475:
Process Process-469:
Process Process-463:
Process Process-461:
Process Process-460:
Process Process-466:
Process Process-478:
Process Process-482:
Process Process-484:
Process Process-476:
Process Process-473:
Process Process-467:
Process Process-462:
Process Process-457:
Process Process-468:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call

  File "/home/pranavgokhale/.conda/envs/QOC_qiskit/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/pranavgokhale/.conda/envs/QOC_qiskit/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/pranavgokhale/.conda/envs/QOC_qiskit/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/pranavgokhale/.conda/envs/QOC_qiskit/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/pranavgokhale/.conda/envs/QOC_qiskit/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/pranavgokhale/.conda/envs/QOC_qiskit/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/pranavgokhale/.conda/envs/QOC_qiskit/lib/python3.6

  File "/home/pranavgokhale/.conda/envs/QOC_qiskit/lib/python3.6/multiprocessing/queues.py", line 93, in get
    with self._rlock:
  File "/home/pranavgokhale/.conda/envs/QOC_qiskit/lib/python3.6/multiprocessing/queues.py", line 93, in get
    with self._rlock:
  File "/home/pranavgokhale/.conda/envs/QOC_qiskit/lib/python3.6/multiprocessing/queues.py", line 93, in get
    with self._rlock:
  File "/home/pranavgokhale/.conda/envs/QOC_qiskit/lib/python3.6/multiprocessing/queues.py", line 94, in get
    res = self._recv_bytes()
  File "/home/pranavgokhale/.conda/envs/QOC_qiskit/lib/python3.6/multiprocessing/queues.py", line 93, in get
    with self._rlock:
  File "/home/pranavgokhale/.conda/envs/QOC_qiskit/lib/python3.6/multiprocessing/queues.py", line 93, in get
    with self._rlock:
  File "/home/pranavgokhale/.conda/envs/QOC_qiskit/lib/python3.6/multiprocessing/queues.py", line 93, in get
    with self._rlock:
  File "/home/pranavgokhale/.conda/envs/QOC_qiskit/lib/python3.6/multiproces

In [16]:
print(times)
print(sum(times))

[1.55, 1.05, 2.15]
4.75


In [9]:
times = []
for slice in slices:
    U = slice.unitary()
    times.append(binary_search_for_shortest_pulse_time(0.0, 10.0, tolerance=0.3))
print(times)
print(sum(times))



trying total_time: 5.0 for unitary of size (4, 4)
data saved at: /project/ftchong/qoc/pranav/00048_Mar28.h5
Using 6 Taylor terms and 1 Scaling & Squaring terms
Building graph:
Propagators initialized.
Operators weight initialized.
Intermediate propagation variables initialized.
Intermediate propagators initialized.
Vectors initialized.
Training loss initialized.
Optimizer initialized.
Utilities initialized.
Graph built!
Initialized
Error = :5.94e-01; Runtime: 0.1s; Iterations = 0, grads =   4.932e-01, unitary_metric = 1.00000
Error = :5.04e-02; Runtime: 1.1s; Iterations = 20, grads =   5.692e-02, unitary_metric = 1.00000
Error = :9.54e-03; Runtime: 1.8s; Iterations = 40, grads =   9.214e-03, unitary_metric = 1.00000
Error = :8.69e-04; Runtime: 2.3s; Iterations = 55, grads =   1.932e-03, unitary_metric = 1.00000
data saved at: /project/ftchong/qoc/pranav/00048_Mar28.h5


trying total_time: 2.5 for unitary of size (4, 4)
data saved at: /project/ftchong/qoc/pranav/00049_Mar28.h5
Using 6

Error = :1.68e-01; Runtime: 3.9s; Iterations = 600, grads =   1.406e-06, unitary_metric = 1.00000
Error = :1.68e-01; Runtime: 4.1s; Iterations = 620, grads =   1.237e-06, unitary_metric = 1.00000
Error = :1.68e-01; Runtime: 4.2s; Iterations = 640, grads =   1.092e-06, unitary_metric = 1.00000
Error = :1.68e-01; Runtime: 4.3s; Iterations = 660, grads =   9.681e-07, unitary_metric = 1.00000
Error = :1.68e-01; Runtime: 4.4s; Iterations = 680, grads =   8.611e-07, unitary_metric = 1.00000
Error = :1.68e-01; Runtime: 4.5s; Iterations = 700, grads =   7.684e-07, unitary_metric = 1.00000
Error = :1.68e-01; Runtime: 4.7s; Iterations = 720, grads =   6.877e-07, unitary_metric = 1.00000
Error = :1.68e-01; Runtime: 4.8s; Iterations = 740, grads =   6.173e-07, unitary_metric = 1.00000
Error = :1.68e-01; Runtime: 4.9s; Iterations = 760, grads =   5.557e-07, unitary_metric = 1.00000
Error = :1.68e-01; Runtime: 5.0s; Iterations = 780, grads =   5.017e-07, unitary_metric = 1.00000
Error = :1.68e-01; R

Error = :1.68e-01; Runtime: 13.5s; Iterations = 2260, grads =   4.096e-08, unitary_metric = 1.00000
Error = :1.68e-01; Runtime: 13.6s; Iterations = 2280, grads =   4.050e-08, unitary_metric = 1.00000
Error = :1.68e-01; Runtime: 13.7s; Iterations = 2300, grads =   4.005e-08, unitary_metric = 1.00000
Error = :1.68e-01; Runtime: 13.8s; Iterations = 2320, grads =   3.961e-08, unitary_metric = 1.00000
Error = :1.68e-01; Runtime: 13.9s; Iterations = 2340, grads =   3.918e-08, unitary_metric = 1.00000
Error = :1.68e-01; Runtime: 14.1s; Iterations = 2360, grads =   3.876e-08, unitary_metric = 1.00000
Error = :1.68e-01; Runtime: 14.2s; Iterations = 2380, grads =   3.834e-08, unitary_metric = 1.00000
Error = :1.68e-01; Runtime: 14.3s; Iterations = 2400, grads =   3.793e-08, unitary_metric = 1.00000
Error = :1.68e-01; Runtime: 14.5s; Iterations = 2420, grads =   3.753e-08, unitary_metric = 1.00000
Error = :1.68e-01; Runtime: 14.6s; Iterations = 2440, grads =   3.714e-08, unitary_metric = 1.00000


Error = :1.68e-01; Runtime: 22.9s; Iterations = 3900, grads =   1.986e-08, unitary_metric = 1.00000
Error = :1.68e-01; Runtime: 23.0s; Iterations = 3920, grads =   1.971e-08, unitary_metric = 1.00000
Error = :1.68e-01; Runtime: 23.1s; Iterations = 3940, grads =   1.955e-08, unitary_metric = 1.00000
Error = :1.68e-01; Runtime: 23.2s; Iterations = 3960, grads =   1.940e-08, unitary_metric = 1.00000
Error = :1.68e-01; Runtime: 23.3s; Iterations = 3980, grads =   1.925e-08, unitary_metric = 1.00000
Error = :1.68e-01; Runtime: 23.4s; Iterations = 4000, grads =   1.910e-08, unitary_metric = 1.00000
Error = :1.68e-01; Runtime: 23.6s; Iterations = 4020, grads =   1.895e-08, unitary_metric = 1.00000
Error = :1.68e-01; Runtime: 23.7s; Iterations = 4040, grads =   1.881e-08, unitary_metric = 1.00000
Error = :1.68e-01; Runtime: 23.8s; Iterations = 4060, grads =   1.867e-08, unitary_metric = 1.00000
Error = :1.68e-01; Runtime: 23.9s; Iterations = 4080, grads =   1.852e-08, unitary_metric = 1.00000


Error = :1.69e-01; Runtime: 32.3s; Iterations = 5540, grads =   1.212e-08, unitary_metric = 1.00000
Error = :1.69e-01; Runtime: 32.4s; Iterations = 5560, grads =   1.205e-08, unitary_metric = 1.00000
Error = :1.69e-01; Runtime: 32.5s; Iterations = 5580, grads =   1.197e-08, unitary_metric = 1.00000
Error = :1.69e-01; Runtime: 32.6s; Iterations = 5600, grads =   1.189e-08, unitary_metric = 1.00000
Error = :1.69e-01; Runtime: 32.7s; Iterations = 5620, grads =   1.181e-08, unitary_metric = 1.00000
Error = :1.69e-01; Runtime: 32.8s; Iterations = 5640, grads =   1.174e-08, unitary_metric = 1.00000
Error = :1.69e-01; Runtime: 32.9s; Iterations = 5660, grads =   1.166e-08, unitary_metric = 1.00000
Error = :1.69e-01; Runtime: 33.0s; Iterations = 5680, grads =   1.158e-08, unitary_metric = 1.00000
Error = :1.69e-01; Runtime: 33.2s; Iterations = 5700, grads =   1.149e-08, unitary_metric = 1.00000
Error = :1.69e-01; Runtime: 33.3s; Iterations = 5720, grads =   1.141e-08, unitary_metric = 1.00000


...

Error = :5.09e-02; Runtime: 114.0s; Iterations = 5840, grads =   5.710e-09, unitary_metric = 1.00000
Error = :5.09e-02; Runtime: 114.4s; Iterations = 5860, grads =   5.624e-09, unitary_metric = 1.00000
Error = :5.09e-02; Runtime: 114.8s; Iterations = 5880, grads =   5.539e-09, unitary_metric = 1.00000
Error = :5.09e-02; Runtime: 115.1s; Iterations = 5900, grads =   5.458e-09, unitary_metric = 1.00000
Error = :5.09e-02; Runtime: 115.6s; Iterations = 5920, grads =   5.376e-09, unitary_metric = 1.00000
Error = :5.09e-02; Runtime: 115.9s; Iterations = 5940, grads =   5.295e-09, unitary_metric = 1.00000
Error = :5.09e-02; Runtime: 116.3s; Iterations = 5960, grads =   5.216e-09, unitary_metric = 1.00000
Error = :5.09e-02; Runtime: 116.7s; Iterations = 5980, grads =   5.139e-09, unitary_metric = 1.00000
Error = :5.09e-02; Runtime: 117.1s; Iterations = 6000, grads =   5.069e-09, unitary_metric = 1.00000
data saved at: /project/ftchong/qoc/pranav/00063_Mar28.h5


trying total_time: 1.85 for uni

Error = :7.96e-03; Runtime: 38.6s; Iterations = 1380, grads =   9.200e-07, unitary_metric = 1.00000
Error = :7.90e-03; Runtime: 39.1s; Iterations = 1400, grads =   8.866e-07, unitary_metric = 1.00000
Error = :7.83e-03; Runtime: 39.7s; Iterations = 1420, grads =   8.548e-07, unitary_metric = 1.00000
Error = :7.78e-03; Runtime: 40.3s; Iterations = 1440, grads =   8.251e-07, unitary_metric = 1.00000
Error = :7.72e-03; Runtime: 40.8s; Iterations = 1460, grads =   7.969e-07, unitary_metric = 1.00000
Error = :7.67e-03; Runtime: 41.3s; Iterations = 1480, grads =   7.704e-07, unitary_metric = 1.00000
Error = :7.61e-03; Runtime: 41.9s; Iterations = 1500, grads =   7.480e-07, unitary_metric = 1.00000
Error = :7.56e-03; Runtime: 42.5s; Iterations = 1520, grads =   7.223e-07, unitary_metric = 1.00000
Error = :7.51e-03; Runtime: 43.0s; Iterations = 1540, grads =   6.991e-07, unitary_metric = 1.00000
Error = :7.47e-03; Runtime: 43.5s; Iterations = 1560, grads =   6.780e-07, unitary_metric = 1.00000


Error = :5.30e-03; Runtime: 83.8s; Iterations = 3020, grads =   1.225e-07, unitary_metric = 1.00000
Error = :5.29e-03; Runtime: 84.3s; Iterations = 3040, grads =   1.193e-07, unitary_metric = 1.00000
Error = :5.28e-03; Runtime: 84.9s; Iterations = 3060, grads =   1.161e-07, unitary_metric = 1.00000
Error = :5.27e-03; Runtime: 85.4s; Iterations = 3080, grads =   1.131e-07, unitary_metric = 1.00000
Error = :5.26e-03; Runtime: 85.9s; Iterations = 3100, grads =   1.102e-07, unitary_metric = 1.00000
Error = :5.26e-03; Runtime: 86.6s; Iterations = 3120, grads =   1.076e-07, unitary_metric = 1.00000
Error = :5.25e-03; Runtime: 87.1s; Iterations = 3140, grads =   1.047e-07, unitary_metric = 1.00000
Error = :5.24e-03; Runtime: 87.6s; Iterations = 3160, grads =   1.020e-07, unitary_metric = 1.00000
Error = :5.23e-03; Runtime: 88.1s; Iterations = 3180, grads =   9.948e-08, unitary_metric = 1.00000
Error = :5.22e-03; Runtime: 88.7s; Iterations = 3200, grads =   9.699e-08, unitary_metric = 1.00000


Error = :4.91e-03; Runtime: 128.9s; Iterations = 4660, grads =   2.469e-08, unitary_metric = 1.00000
Error = :4.91e-03; Runtime: 129.5s; Iterations = 4680, grads =   2.436e-08, unitary_metric = 1.00000
Error = :4.91e-03; Runtime: 130.0s; Iterations = 4700, grads =   2.404e-08, unitary_metric = 1.00000
Error = :4.90e-03; Runtime: 130.6s; Iterations = 4720, grads =   2.382e-08, unitary_metric = 1.00000
Error = :4.90e-03; Runtime: 131.1s; Iterations = 4740, grads =   2.341e-08, unitary_metric = 1.00000
Error = :4.90e-03; Runtime: 131.7s; Iterations = 4760, grads =   2.311e-08, unitary_metric = 1.00000
Error = :4.89e-03; Runtime: 132.2s; Iterations = 4780, grads =   2.281e-08, unitary_metric = 1.00000
Error = :4.89e-03; Runtime: 132.7s; Iterations = 4800, grads =   2.252e-08, unitary_metric = 1.00000
Error = :4.89e-03; Runtime: 133.4s; Iterations = 4820, grads =   2.224e-08, unitary_metric = 1.00000
Error = :4.89e-03; Runtime: 133.9s; Iterations = 4840, grads =   2.196e-08, unitary_metric 

Error = :1.56e-02; Runtime: 6.1s; Iterations = 180, grads =   5.066e-05, unitary_metric = 1.00000
Error = :1.30e-02; Runtime: 6.7s; Iterations = 200, grads =   3.771e-05, unitary_metric = 1.00000
Error = :1.11e-02; Runtime: 7.4s; Iterations = 220, grads =   2.880e-05, unitary_metric = 1.00000
Error = :9.54e-03; Runtime: 8.0s; Iterations = 240, grads =   2.259e-05, unitary_metric = 1.00000
Error = :8.32e-03; Runtime: 8.6s; Iterations = 260, grads =   1.816e-05, unitary_metric = 1.00000
Error = :7.33e-03; Runtime: 9.3s; Iterations = 280, grads =   1.492e-05, unitary_metric = 1.00000
Error = :6.51e-03; Runtime: 9.9s; Iterations = 300, grads =   1.247e-05, unitary_metric = 1.00000
Error = :5.82e-03; Runtime: 10.6s; Iterations = 320, grads =   1.057e-05, unitary_metric = 1.00000
Error = :5.24e-03; Runtime: 11.2s; Iterations = 340, grads =   9.071e-06, unitary_metric = 1.00000
Error = :4.74e-03; Runtime: 11.8s; Iterations = 360, grads =   7.857e-06, unitary_metric = 1.00000
Error = :4.30e-03

In [10]:
times = []
for slice in slices:
    U = slice.unitary()
    times.append(binary_search_for_shortest_pulse_time(0.0, 10.0, tolerance=0.3))
print(times)
print(sum(times))



trying total_time: 5.0 for unitary of size (4, 4)
data saved at: /project/ftchong/qoc/pranav/00066_Mar28.h5
Using 6 Taylor terms and 1 Scaling & Squaring terms
Building graph:
Propagators initialized.
Operators weight initialized.
Intermediate propagation variables initialized.
Intermediate propagators initialized.
Vectors initialized.
Training loss initialized.
Optimizer initialized.
Utilities initialized.
Graph built!
Initialized
Error = :6.20e-01; Runtime: 0.3s; Iterations = 0, grads =   5.153e-01, unitary_metric = 1.00000
Error = :5.18e-02; Runtime: 2.2s; Iterations = 20, grads =   6.785e-02, unitary_metric = 1.00000
Error = :1.32e-02; Runtime: 3.6s; Iterations = 40, grads =   1.567e-02, unitary_metric = 1.00000
Error = :4.93e-04; Runtime: 4.5s; Iterations = 54, grads =   1.886e-03, unitary_metric = 1.00000
data saved at: /project/ftchong/qoc/pranav/00066_Mar28.h5


trying total_time: 2.5 for unitary of size (4, 4)
data saved at: /project/ftchong/qoc/pranav/00067_Mar28.h5
Using 6

Error = :1.68e-01; Runtime: 6.1s; Iterations = 560, grads =   7.449e-07, unitary_metric = 1.00000
Error = :1.68e-01; Runtime: 6.3s; Iterations = 580, grads =   6.474e-07, unitary_metric = 1.00000
Error = :1.68e-01; Runtime: 6.5s; Iterations = 600, grads =   5.699e-07, unitary_metric = 1.00000
Error = :1.68e-01; Runtime: 6.8s; Iterations = 620, grads =   5.073e-07, unitary_metric = 1.00000
Error = :1.68e-01; Runtime: 7.0s; Iterations = 640, grads =   4.561e-07, unitary_metric = 1.00000
Error = :1.68e-01; Runtime: 7.2s; Iterations = 660, grads =   4.137e-07, unitary_metric = 1.00000
Error = :1.68e-01; Runtime: 7.4s; Iterations = 680, grads =   3.779e-07, unitary_metric = 1.00000
Error = :1.68e-01; Runtime: 7.6s; Iterations = 700, grads =   3.475e-07, unitary_metric = 1.00000
Error = :1.68e-01; Runtime: 7.9s; Iterations = 720, grads =   3.211e-07, unitary_metric = 1.00000
Error = :1.68e-01; Runtime: 8.0s; Iterations = 740, grads =   2.981e-07, unitary_metric = 1.00000
Error = :1.68e-01; R

Error = :1.68e-01; Runtime: 23.8s; Iterations = 2220, grads =   2.186e-08, unitary_metric = 1.00000
Error = :1.68e-01; Runtime: 24.0s; Iterations = 2240, grads =   2.160e-08, unitary_metric = 1.00000
Error = :1.68e-01; Runtime: 24.2s; Iterations = 2260, grads =   2.136e-08, unitary_metric = 1.00000
Error = :1.68e-01; Runtime: 24.4s; Iterations = 2280, grads =   2.112e-08, unitary_metric = 1.00000
Error = :1.68e-01; Runtime: 24.6s; Iterations = 2300, grads =   2.089e-08, unitary_metric = 1.00000
Error = :1.68e-01; Runtime: 24.9s; Iterations = 2320, grads =   2.066e-08, unitary_metric = 1.00000
Error = :1.68e-01; Runtime: 25.1s; Iterations = 2340, grads =   2.045e-08, unitary_metric = 1.00000
Error = :1.68e-01; Runtime: 25.3s; Iterations = 2360, grads =   2.025e-08, unitary_metric = 1.00000
Error = :1.68e-01; Runtime: 25.5s; Iterations = 2380, grads =   2.005e-08, unitary_metric = 1.00000
Error = :1.68e-01; Runtime: 25.7s; Iterations = 2400, grads =   1.985e-08, unitary_metric = 1.00000


Error = :1.68e-01; Runtime: 41.2s; Iterations = 3860, grads =   1.270e-08, unitary_metric = 1.00000
Error = :1.68e-01; Runtime: 41.4s; Iterations = 3880, grads =   1.265e-08, unitary_metric = 1.00000
Error = :1.68e-01; Runtime: 41.6s; Iterations = 3900, grads =   1.259e-08, unitary_metric = 1.00000
Error = :1.68e-01; Runtime: 41.9s; Iterations = 3920, grads =   1.253e-08, unitary_metric = 1.00000
Error = :1.68e-01; Runtime: 42.1s; Iterations = 3940, grads =   1.248e-08, unitary_metric = 1.00000
Error = :1.68e-01; Runtime: 42.3s; Iterations = 3960, grads =   1.242e-08, unitary_metric = 1.00000
Error = :1.68e-01; Runtime: 42.5s; Iterations = 3980, grads =   1.237e-08, unitary_metric = 1.00000
Error = :1.68e-01; Runtime: 42.7s; Iterations = 4000, grads =   1.231e-08, unitary_metric = 1.00000
Error = :1.68e-01; Runtime: 42.9s; Iterations = 4020, grads =   1.226e-08, unitary_metric = 1.00000
Error = :1.68e-01; Runtime: 43.1s; Iterations = 4040, grads =   1.221e-08, unitary_metric = 1.00000


Error = :1.69e-01; Runtime: 58.6s; Iterations = 5480, grads =   8.656e-09, unitary_metric = 1.00000
Error = :1.69e-01; Runtime: 58.8s; Iterations = 5500, grads =   8.601e-09, unitary_metric = 1.00000
Error = :1.69e-01; Runtime: 59.1s; Iterations = 5520, grads =   8.544e-09, unitary_metric = 1.00000
Error = :1.69e-01; Runtime: 59.3s; Iterations = 5540, grads =   8.489e-09, unitary_metric = 1.00000
Error = :1.69e-01; Runtime: 59.6s; Iterations = 5560, grads =   8.432e-09, unitary_metric = 1.00000
Error = :1.69e-01; Runtime: 59.8s; Iterations = 5580, grads =   8.374e-09, unitary_metric = 1.00000
Error = :1.69e-01; Runtime: 60.0s; Iterations = 5600, grads =   8.316e-09, unitary_metric = 1.00000
Error = :1.69e-01; Runtime: 60.3s; Iterations = 5620, grads =   8.257e-09, unitary_metric = 1.00000
Error = :1.69e-01; Runtime: 60.5s; Iterations = 5640, grads =   8.200e-09, unitary_metric = 1.00000
Error = :1.69e-01; Runtime: 60.7s; Iterations = 5660, grads =   8.139e-09, unitary_metric = 1.00000


...

Error = :5.90e-02; Runtime: 110.7s; Iterations = 5700, grads =   1.189e-08, unitary_metric = 1.00000
Error = :5.90e-02; Runtime: 111.1s; Iterations = 5720, grads =   1.174e-08, unitary_metric = 1.00000
Error = :5.90e-02; Runtime: 111.5s; Iterations = 5740, grads =   1.159e-08, unitary_metric = 1.00000
Error = :5.90e-02; Runtime: 111.9s; Iterations = 5760, grads =   1.144e-08, unitary_metric = 1.00000
Error = :5.90e-02; Runtime: 112.2s; Iterations = 5780, grads =   1.130e-08, unitary_metric = 1.00000
Error = :5.90e-02; Runtime: 112.6s; Iterations = 5800, grads =   1.116e-08, unitary_metric = 1.00000
Error = :5.90e-02; Runtime: 113.1s; Iterations = 5820, grads =   1.102e-08, unitary_metric = 1.00000
Error = :5.90e-02; Runtime: 113.5s; Iterations = 5840, grads =   1.089e-08, unitary_metric = 1.00000
Error = :5.90e-02; Runtime: 113.8s; Iterations = 5860, grads =   1.075e-08, unitary_metric = 1.00000
Error = :5.90e-02; Runtime: 114.2s; Iterations = 5880, grads =   1.062e-08, unitary_metric 

Error = :7.36e-03; Runtime: 34.5s; Iterations = 1240, grads =   1.120e-06, unitary_metric = 1.00000
Error = :7.30e-03; Runtime: 35.1s; Iterations = 1260, grads =   1.062e-06, unitary_metric = 1.00000
Error = :7.25e-03; Runtime: 35.6s; Iterations = 1280, grads =   1.007e-06, unitary_metric = 1.00000
Error = :7.20e-03; Runtime: 36.1s; Iterations = 1300, grads =   9.569e-07, unitary_metric = 1.00000
Error = :7.15e-03; Runtime: 36.7s; Iterations = 1320, grads =   9.102e-07, unitary_metric = 1.00000
Error = :7.11e-03; Runtime: 37.3s; Iterations = 1340, grads =   8.670e-07, unitary_metric = 1.00000
Error = :7.06e-03; Runtime: 37.8s; Iterations = 1360, grads =   8.296e-07, unitary_metric = 1.00000
Error = :7.02e-03; Runtime: 38.3s; Iterations = 1380, grads =   7.897e-07, unitary_metric = 1.00000
Error = :6.98e-03; Runtime: 38.9s; Iterations = 1400, grads =   7.547e-07, unitary_metric = 1.00000
Error = :6.94e-03; Runtime: 39.5s; Iterations = 1420, grads =   7.223e-07, unitary_metric = 1.00000


Error = :5.77e-03; Runtime: 79.5s; Iterations = 2880, grads =   1.110e-07, unitary_metric = 1.00000
Error = :5.76e-03; Runtime: 80.1s; Iterations = 2900, grads =   1.088e-07, unitary_metric = 1.00000
Error = :5.75e-03; Runtime: 80.7s; Iterations = 2920, grads =   1.067e-07, unitary_metric = 1.00000
Error = :5.74e-03; Runtime: 81.2s; Iterations = 2940, grads =   1.047e-07, unitary_metric = 1.00000
Error = :5.74e-03; Runtime: 81.7s; Iterations = 2960, grads =   1.027e-07, unitary_metric = 1.00000
Error = :5.73e-03; Runtime: 82.3s; Iterations = 2980, grads =   1.008e-07, unitary_metric = 1.00000
Error = :5.72e-03; Runtime: 82.8s; Iterations = 3000, grads =   9.892e-08, unitary_metric = 1.00000
Error = :5.72e-03; Runtime: 83.4s; Iterations = 3020, grads =   9.706e-08, unitary_metric = 1.00000
Error = :5.71e-03; Runtime: 84.0s; Iterations = 3040, grads =   9.557e-08, unitary_metric = 1.00000
Error = :5.70e-03; Runtime: 84.5s; Iterations = 3060, grads =   9.364e-08, unitary_metric = 1.00000


Error = :5.44e-03; Runtime: 124.8s; Iterations = 4520, grads =   3.057e-08, unitary_metric = 1.00000
Error = :5.44e-03; Runtime: 125.3s; Iterations = 4540, grads =   3.040e-08, unitary_metric = 1.00000
Error = :5.44e-03; Runtime: 125.8s; Iterations = 4560, grads =   3.028e-08, unitary_metric = 1.00000
Error = :5.44e-03; Runtime: 126.3s; Iterations = 4580, grads =   3.021e-08, unitary_metric = 1.00000
Error = :5.45e-03; Runtime: 126.9s; Iterations = 4600, grads =   3.021e-08, unitary_metric = 1.00000
Error = :5.45e-03; Runtime: 127.5s; Iterations = 4620, grads =   3.029e-08, unitary_metric = 1.00000
Error = :5.46e-03; Runtime: 128.0s; Iterations = 4640, grads =   3.046e-08, unitary_metric = 1.00000
Error = :5.46e-03; Runtime: 128.6s; Iterations = 4660, grads =   3.074e-08, unitary_metric = 1.00000
Error = :5.47e-03; Runtime: 129.1s; Iterations = 4680, grads =   3.117e-08, unitary_metric = 1.00000
Error = :5.48e-03; Runtime: 129.6s; Iterations = 4700, grads =   3.176e-08, unitary_metric 

Error = :2.86e-01; Runtime: 1.6s; Iterations = 40, grads =   5.132e-02, unitary_metric = 1.00000
Error = :1.21e-01; Runtime: 2.2s; Iterations = 60, grads =   7.538e-03, unitary_metric = 1.00000
Error = :6.54e-02; Runtime: 2.8s; Iterations = 80, grads =   1.500e-03, unitary_metric = 1.00000
Error = :5.08e-02; Runtime: 3.5s; Iterations = 100, grads =   4.319e-04, unitary_metric = 1.00000
Error = :4.16e-02; Runtime: 4.2s; Iterations = 120, grads =   2.934e-04, unitary_metric = 1.00000
Error = :3.35e-02; Runtime: 4.8s; Iterations = 140, grads =   1.850e-04, unitary_metric = 1.00000
Error = :2.70e-02; Runtime: 5.4s; Iterations = 160, grads =   1.207e-04, unitary_metric = 1.00000
Error = :2.18e-02; Runtime: 6.0s; Iterations = 180, grads =   8.613e-05, unitary_metric = 1.00000
Error = :1.78e-02; Runtime: 6.6s; Iterations = 200, grads =   6.354e-05, unitary_metric = 1.00000
Error = :1.47e-02; Runtime: 7.3s; Iterations = 220, grads =   4.762e-05, unitary_metric = 1.00000
Error = :1.23e-02; Runt

In [11]:
times = []
for slice in slices:
    U = slice.unitary()
    times.append(binary_search_for_shortest_pulse_time(0.0, 10.0, tolerance=0.3))
print(times)
print(sum(times))



trying total_time: 5.0 for unitary of size (4, 4)
data saved at: /project/ftchong/qoc/pranav/00084_Mar28.h5
Using 6 Taylor terms and 1 Scaling & Squaring terms
Building graph:
Propagators initialized.
Operators weight initialized.
Intermediate propagation variables initialized.
Intermediate propagators initialized.
Vectors initialized.
Training loss initialized.
Optimizer initialized.
Utilities initialized.
Graph built!
Initialized
Error = :7.61e-01; Runtime: 0.3s; Iterations = 0, grads =   7.521e-01, unitary_metric = 1.00000
Error = :3.92e-02; Runtime: 2.2s; Iterations = 20, grads =   3.536e-02, unitary_metric = 1.00000
Error = :9.56e-03; Runtime: 3.5s; Iterations = 40, grads =   2.359e-02, unitary_metric = 1.00000
Error = :7.48e-04; Runtime: 4.7s; Iterations = 57, grads =   3.967e-03, unitary_metric = 1.00000
data saved at: /project/ftchong/qoc/pranav/00084_Mar28.h5


trying total_time: 2.5 for unitary of size (4, 4)
data saved at: /project/ftchong/qoc/pranav/00085_Mar28.h5
Using 6

Error = :1.69e-01; Runtime: 5.9s; Iterations = 540, grads =   4.559e-06, unitary_metric = 1.00000
Error = :1.69e-01; Runtime: 6.1s; Iterations = 560, grads =   3.910e-06, unitary_metric = 1.00000
Error = :1.69e-01; Runtime: 6.5s; Iterations = 580, grads =   3.368e-06, unitary_metric = 1.00000
Error = :1.69e-01; Runtime: 6.7s; Iterations = 600, grads =   2.914e-06, unitary_metric = 1.00000
Error = :1.69e-01; Runtime: 6.9s; Iterations = 620, grads =   2.533e-06, unitary_metric = 1.00000
Error = :1.69e-01; Runtime: 7.1s; Iterations = 640, grads =   2.213e-06, unitary_metric = 1.00000
Error = :1.69e-01; Runtime: 7.3s; Iterations = 660, grads =   1.941e-06, unitary_metric = 1.00000
Error = :1.68e-01; Runtime: 7.6s; Iterations = 680, grads =   1.711e-06, unitary_metric = 1.00000
Error = :1.68e-01; Runtime: 7.7s; Iterations = 700, grads =   1.514e-06, unitary_metric = 1.00000
Error = :1.68e-01; Runtime: 8.0s; Iterations = 720, grads =   1.345e-06, unitary_metric = 1.00000
Error = :1.68e-01; R

Error = :1.68e-01; Runtime: 23.6s; Iterations = 2200, grads =   3.728e-08, unitary_metric = 1.00000
Error = :1.68e-01; Runtime: 23.9s; Iterations = 2220, grads =   3.684e-08, unitary_metric = 1.00000
Error = :1.68e-01; Runtime: 24.1s; Iterations = 2240, grads =   3.641e-08, unitary_metric = 1.00000
Error = :1.68e-01; Runtime: 24.3s; Iterations = 2260, grads =   3.599e-08, unitary_metric = 1.00000
Error = :1.68e-01; Runtime: 24.5s; Iterations = 2280, grads =   3.558e-08, unitary_metric = 1.00000
Error = :1.68e-01; Runtime: 24.7s; Iterations = 2300, grads =   3.518e-08, unitary_metric = 1.00000
Error = :1.68e-01; Runtime: 24.9s; Iterations = 2320, grads =   3.479e-08, unitary_metric = 1.00000
Error = :1.68e-01; Runtime: 25.1s; Iterations = 2340, grads =   3.441e-08, unitary_metric = 1.00000
Error = :1.68e-01; Runtime: 25.3s; Iterations = 2360, grads =   3.403e-08, unitary_metric = 1.00000
Error = :1.68e-01; Runtime: 25.5s; Iterations = 2380, grads =   3.367e-08, unitary_metric = 1.00000


Error = :1.68e-01; Runtime: 41.0s; Iterations = 3840, grads =   1.824e-08, unitary_metric = 1.00000
Error = :1.68e-01; Runtime: 41.2s; Iterations = 3860, grads =   1.811e-08, unitary_metric = 1.00000
Error = :1.68e-01; Runtime: 41.4s; Iterations = 3880, grads =   1.799e-08, unitary_metric = 1.00000
Error = :1.68e-01; Runtime: 41.6s; Iterations = 3900, grads =   1.786e-08, unitary_metric = 1.00000
Error = :1.68e-01; Runtime: 41.9s; Iterations = 3920, grads =   1.773e-08, unitary_metric = 1.00000
Error = :1.68e-01; Runtime: 42.1s; Iterations = 3940, grads =   1.761e-08, unitary_metric = 1.00000
Error = :1.68e-01; Runtime: 42.3s; Iterations = 3960, grads =   1.748e-08, unitary_metric = 1.00000
Error = :1.68e-01; Runtime: 42.5s; Iterations = 3980, grads =   1.736e-08, unitary_metric = 1.00000
Error = :1.68e-01; Runtime: 42.7s; Iterations = 4000, grads =   1.723e-08, unitary_metric = 1.00000
Error = :1.68e-01; Runtime: 43.0s; Iterations = 4020, grads =   1.711e-08, unitary_metric = 1.00000


Error = :1.68e-01; Runtime: 58.4s; Iterations = 5480, grads =   1.080e-08, unitary_metric = 1.00000
Error = :1.68e-01; Runtime: 58.6s; Iterations = 5500, grads =   1.075e-08, unitary_metric = 1.00000
Error = :1.68e-01; Runtime: 58.9s; Iterations = 5520, grads =   1.070e-08, unitary_metric = 1.00000
Error = :1.68e-01; Runtime: 59.1s; Iterations = 5540, grads =   1.065e-08, unitary_metric = 1.00000
Error = :1.68e-01; Runtime: 59.3s; Iterations = 5560, grads =   1.060e-08, unitary_metric = 1.00000
Error = :1.68e-01; Runtime: 59.5s; Iterations = 5580, grads =   1.055e-08, unitary_metric = 1.00000
Error = :1.69e-01; Runtime: 59.7s; Iterations = 5600, grads =   1.050e-08, unitary_metric = 1.00000
Error = :1.69e-01; Runtime: 59.9s; Iterations = 5620, grads =   1.045e-08, unitary_metric = 1.00000
Error = :1.69e-01; Runtime: 60.1s; Iterations = 5640, grads =   1.041e-08, unitary_metric = 1.00000
Error = :1.69e-01; Runtime: 60.3s; Iterations = 5660, grads =   1.036e-08, unitary_metric = 1.00000


...

Error = :5.81e-02; Runtime: 110.6s; Iterations = 5700, grads =   1.065e-08, unitary_metric = 1.00000
Error = :5.81e-02; Runtime: 111.1s; Iterations = 5720, grads =   1.054e-08, unitary_metric = 1.00000
Error = :5.81e-02; Runtime: 111.4s; Iterations = 5740, grads =   1.043e-08, unitary_metric = 1.00000
Error = :5.81e-02; Runtime: 111.8s; Iterations = 5760, grads =   1.033e-08, unitary_metric = 1.00000
Error = :5.81e-02; Runtime: 112.2s; Iterations = 5780, grads =   1.022e-08, unitary_metric = 1.00000
Error = :5.81e-02; Runtime: 112.6s; Iterations = 5800, grads =   1.009e-08, unitary_metric = 1.00000
Error = :5.81e-02; Runtime: 113.0s; Iterations = 5820, grads =   9.988e-09, unitary_metric = 1.00000
Error = :5.81e-02; Runtime: 113.4s; Iterations = 5840, grads =   9.919e-09, unitary_metric = 1.00000
Error = :5.81e-02; Runtime: 113.8s; Iterations = 5860, grads =   9.925e-09, unitary_metric = 1.00000
Error = :5.81e-02; Runtime: 114.1s; Iterations = 5880, grads =   9.986e-09, unitary_metric 

Error = :3.75e-03; Runtime: 34.7s; Iterations = 1240, grads =   5.485e-07, unitary_metric = 1.00000
Error = :3.72e-03; Runtime: 35.3s; Iterations = 1260, grads =   5.247e-07, unitary_metric = 1.00000
Error = :3.69e-03; Runtime: 35.8s; Iterations = 1280, grads =   5.022e-07, unitary_metric = 1.00000
Error = :3.66e-03; Runtime: 36.3s; Iterations = 1300, grads =   4.809e-07, unitary_metric = 1.00000
Error = :3.64e-03; Runtime: 36.9s; Iterations = 1320, grads =   4.608e-07, unitary_metric = 1.00000
Error = :3.61e-03; Runtime: 37.5s; Iterations = 1340, grads =   4.418e-07, unitary_metric = 1.00000
Error = :3.58e-03; Runtime: 38.0s; Iterations = 1360, grads =   4.237e-07, unitary_metric = 1.00000
Error = :3.56e-03; Runtime: 38.5s; Iterations = 1380, grads =   4.066e-07, unitary_metric = 1.00000
Error = :3.54e-03; Runtime: 39.1s; Iterations = 1400, grads =   3.904e-07, unitary_metric = 1.00000
Error = :3.51e-03; Runtime: 39.7s; Iterations = 1420, grads =   3.750e-07, unitary_metric = 1.00000


Error = :2.81e-03; Runtime: 80.1s; Iterations = 2880, grads =   5.294e-08, unitary_metric = 1.00000
Error = :2.80e-03; Runtime: 80.7s; Iterations = 2900, grads =   5.203e-08, unitary_metric = 1.00000
Error = :2.80e-03; Runtime: 81.3s; Iterations = 2920, grads =   5.114e-08, unitary_metric = 1.00000
Error = :2.79e-03; Runtime: 81.8s; Iterations = 2940, grads =   5.028e-08, unitary_metric = 1.00000
Error = :2.79e-03; Runtime: 82.4s; Iterations = 2960, grads =   4.946e-08, unitary_metric = 1.00000
Error = :2.78e-03; Runtime: 82.9s; Iterations = 2980, grads =   4.866e-08, unitary_metric = 1.00000
Error = :2.78e-03; Runtime: 83.4s; Iterations = 3000, grads =   4.789e-08, unitary_metric = 1.00000
Error = :2.78e-03; Runtime: 84.0s; Iterations = 3020, grads =   4.714e-08, unitary_metric = 1.00000
Error = :2.77e-03; Runtime: 84.6s; Iterations = 3040, grads =   4.643e-08, unitary_metric = 1.00000
Error = :2.77e-03; Runtime: 85.1s; Iterations = 3060, grads =   4.573e-08, unitary_metric = 1.00000


Error = :2.55e-03; Runtime: 125.4s; Iterations = 4520, grads =   4.972e-08, unitary_metric = 1.00000
Error = :2.55e-03; Runtime: 126.0s; Iterations = 4540, grads =   4.895e-08, unitary_metric = 1.00000
Error = :2.54e-03; Runtime: 126.5s; Iterations = 4560, grads =   4.821e-08, unitary_metric = 1.00000
Error = :2.54e-03; Runtime: 127.0s; Iterations = 4580, grads =   4.749e-08, unitary_metric = 1.00000
Error = :2.54e-03; Runtime: 127.5s; Iterations = 4600, grads =   4.679e-08, unitary_metric = 1.00000
Error = :2.53e-03; Runtime: 128.2s; Iterations = 4620, grads =   4.611e-08, unitary_metric = 1.00000
Error = :2.53e-03; Runtime: 128.7s; Iterations = 4640, grads =   4.545e-08, unitary_metric = 1.00000
Error = :2.52e-03; Runtime: 129.2s; Iterations = 4660, grads =   4.481e-08, unitary_metric = 1.00000
Error = :2.52e-03; Runtime: 129.8s; Iterations = 4680, grads =   4.418e-08, unitary_metric = 1.00000
Error = :2.52e-03; Runtime: 130.3s; Iterations = 4700, grads =   4.356e-08, unitary_metric 

Error = :2.70e-01; Runtime: 1.6s; Iterations = 40, grads =   3.200e-02, unitary_metric = 1.00000
Error = :1.18e-01; Runtime: 2.2s; Iterations = 60, grads =   9.032e-03, unitary_metric = 1.00000
Error = :5.95e-02; Runtime: 2.9s; Iterations = 80, grads =   1.362e-03, unitary_metric = 1.00000
Error = :4.19e-02; Runtime: 3.5s; Iterations = 100, grads =   4.370e-04, unitary_metric = 1.00000
Error = :3.16e-02; Runtime: 4.1s; Iterations = 120, grads =   2.464e-04, unitary_metric = 1.00000
Error = :2.42e-02; Runtime: 4.8s; Iterations = 140, grads =   1.231e-04, unitary_metric = 1.00000
Error = :1.92e-02; Runtime: 5.5s; Iterations = 160, grads =   7.567e-05, unitary_metric = 1.00000
Error = :1.55e-02; Runtime: 6.1s; Iterations = 180, grads =   5.540e-05, unitary_metric = 1.00000
Error = :1.28e-02; Runtime: 6.7s; Iterations = 200, grads =   4.187e-05, unitary_metric = 1.00000
Error = :1.07e-02; Runtime: 7.4s; Iterations = 220, grads =   3.184e-05, unitary_metric = 1.00000
Error = :9.11e-03; Runt